In [1]:
import dill

In [2]:
Mv_location = f"./../data/GEO/preprocessed/training.dill"
print(f"loading Mv, meta, mapping from {Mv_location}")
Mv, meta = dill.load(open(Mv_location, 'rb'))
print(Mv.shape, meta.shape)

loading Mv, meta, mapping from ./../data/GEO/preprocessed/training.dill
(10351, 297598) (10351, 10)


In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

In [10]:
sampled_samples = {}

for n in [100, 500, 1000, 5000, 10000]:
    print(n)
    sampled_index = []

    stratified_splitter = StratifiedShuffleSplit(n_splits=10, test_size=n, random_state=9)  # More splits for better distribution

    X = Mv  
    y = meta['training.ID']  

    for train_index, test_index in stratified_splitter.split(X, y):
        sampled_index = list(X.iloc[test_index].index)

        # Check if all classes are included
        if len(set(y.iloc[test_index])) == len(set(y)):  
            sampled_samples[n] = sampled_index
            break  # Stop early if all classes are represented
    
    # If not all classes are included, force at least one sample per class
    if len(set(y.loc[sampled_index])) < len(set(y)):
        print(f"Warning: Not all classes are represented in n={n}, adjusting...")

        class_samples = meta.groupby('training.ID').sample(n=1, random_state=9).index.tolist()
        remaining_needed = n - len(class_samples)

        additional_samples = list(set(sampled_index) - set(class_samples))[:remaining_needed]
        sampled_samples[n] = class_samples + additional_samples

100
500
1000
5000
10000


In [12]:
[(len(idx), meta.loc[idx]['training.ID'].nunique()) for n, idx in sampled_samples.items()]

[(100, 55), (500, 55), (1000, 55), (5000, 55), (10000, 55)]

In [13]:
with open('./time/sampled_samples.dill', 'wb') as f:
    dill.dump(sampled_samples, f)

In [6]:
# sampled_samples = dict()
# # Define the stratified splitter
# stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=int(n/fold_Mvs[f].shape[0]), random_state=9)
# samples = list()

# # Specify the features and the target variable
# X = fold_Mvs[f][0]  # Features
# y = fold_Mvs[f][1]['tissue_name']  # Target variable (metadata column)

# # Perform the stratified split
# for train_index, test_index in stratified_splitter.split(X, y):
#     sampled_index = list(X.iloc[test_index].index)
#     samples.append(sampled_index)

# sampled_samples[n] = samples


### over different number of samples